# Una solución empresarial completa

## Ahora llevaremos nuestro proyecto del día 1 al siguiente nivel

### DESAFÍO EMPRESARIAL:

Crear un producto que genere un folleto para una empresa que se utilizará para posibles clientes, inversores y posibles reclutas.

Se nos proporcionará un nombre de empresa y su sitio web principal.

Consulte el final de este cuaderno para ver ejemplos de aplicaciones empresariales del mundo real.

Y recuerde: ¡siempre estoy disponible si tiene problemas o ideas! No dude en comunicarse conmigo.

In [1]:
# imports
# Si esto falla, verifica que esté ejecutándose desde un entorno "activado" con (llms) en el símbolo del sistema

import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [2]:
# Inicialización y constantes and constants

load_dotenv()
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key[:8]=='sk-proj-':
    print("La clave de API parece buena")
else:
    print("¿Puede haber un problema con tu clave API? ¡Visita el cuaderno de resolución de problemas!")
    
MODEL = 'gpt-4o-mini'
openai = OpenAI()

La clave de API parece buena


In [3]:
# La clase para representar una Página Web

class Website:
    """
    Una clase de utilidad para representar un sitio web que hemos scrappeado, ahora con enlaces
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "Sin título"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Título de la Web:\n{self.title}\nContenido de la Web:\n{self.text}\n\n"

In [ ]:
frog = Website("https://cursos.frogamesformacion.com")
print(frog.get_contents())
frog.links

## Primer paso: hacer que GPT-4o-mini determine qué enlaces son relevantes

### Usar una llamada a gpt-4o-mini para leer los enlaces en una página web y responder en JSON estructurado.
Debería decidir qué enlaces son relevantes y reemplazar los enlaces relativos como "/about" con "https://company.com/about".
Usaremos "one shot prompting" en las que proporcionamos un ejemplo de cómo debería responder en la solicitud.

Este es un excelente caso de uso para un LLM, porque requiere una comprensión matizada. Imagínate intentar programar esto sin LLMs analizando la página web: ¡sería muy difícil!

Nota al margen: existe una técnica más avanzada llamada "Salidas estructuradas" en la que requerimos que el modelo responda de acuerdo con una especificación. Cubrimos esta técnica en la Semana 8 durante nuestro proyecto autónomo de inteligencia artificial Agentic.

In [14]:
link_system_prompt = "Se te proporciona una lista de enlaces que se encuentran en una página web. \
Puedes decidir cuáles de los enlaces serían los más relevantes para incluir en un folleto sobre la empresa, \
como enlaces a una página Acerca de, una página de la empresa, las carreras/empleos disponibles o páginas de Cursos/Packs.\n"
link_system_prompt += "Debes responder en JSON como en este ejemplo:"
link_system_prompt += """
{
    "links": [
        {"type": "Pagina Sobre nosotros", "url": "https://url.completa/aqui/va/sobre/nosotros"},
        {"type": "Pagina de Cursos": "url": "https://otra.url.completa/courses"}
    ]
}
"""

In [15]:
print(link_system_prompt)

Se te proporciona una lista de enlaces que se encuentran en una página web. Puedes decidir cuáles de los enlaces serían los más relevantes para incluir en un folleto sobre la empresa, como enlaces a una página Acerca de, una página de la empresa, las carreras/empleos disponibles o páginas de Cursos/Packs.
Debes responder en JSON como en este ejemplo:
{
    "links": [
        {"type": "Pagina Sobre nosotros", "url": "https://url.completa/aqui/va/sobre/nosotros"},
        {"type": "Pagina de Cursos": "url": "https://otra.url.completa/courses"}
    ]
}



In [8]:
def get_links_user_prompt(website):
    user_prompt = f"Aquí hay una lista de enlaces de la página web {website.url} - "
    user_prompt += "Por favor, decide cuáles de estos son enlaces web relevantes para un folleto sobre la empresa. Responde con la URL https completa en formato JSON. \
No incluyas Términos y Condiciones, Privacidad ni enlaces de correo electrónico.\n"
    user_prompt += "Links (puede que algunos sean links relativos):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [9]:
print(get_links_user_prompt(frog))

Aquí hay una lista de enlaces de la página web https://cursos.frogamesformacion.com - Por favor, decide cuáles de estos son enlaces web relevantes para un folleto sobre la empresa. Responde con la URL https completa en formato JSON. No incluyas Términos y Condiciones, Privacidad ni enlaces de correo electrónico.
Links (puede que algunos sean links relativos):
#main-content
/
/
/users/sign_in
/collections
https://cursos.frogamesformacion.com/pages/rutas
https://cursos.frogamesformacion.com/pages/instructores
https://cursos.frogamesformacion.com/pages/certificaciones
https://cursos.frogamesformacion.com/collections
/courses/matematicas-ml-2
/courses/matematicas-ml-2
/courses/obsidian
/courses/obsidian
/courses/trigonometria-desde-cero
/courses/trigonometria-desde-cero
/courses/kotlin-desde-cero
/courses/kotlin-desde-cero
/courses/matematicas-ml
/courses/matematicas-ml
/courses/github-android
/courses/github-android
/courses/presencia-redes-sociales
/courses/presencia-redes-sociales
/cour

In [10]:
def get_links(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)

In [11]:
anthropic = Website("https://anthropic.com")
anthropic.links

['/',
 '/claude',
 '/team',
 '/enterprise',
 '/api',
 '/pricing',
 '/research',
 '/company',
 '/careers',
 '/news',
 'https://www.anthropic.com/research#entry:8@1:url',
 'https://www.anthropic.com/claude',
 'https://claude.ai/',
 '/api',
 '/news/3-5-models-and-computer-use',
 '/claude/sonnet',
 '/claude/haiku',
 '/news/claude-for-enterprise',
 '/research/constitutional-ai-harmlessness-from-ai-feedback',
 '/news/core-views-on-ai-safety',
 '/jobs',
 '/',
 '/claude',
 '/api',
 '/team',
 '/pricing',
 '/research',
 '/company',
 '/customers',
 '/news',
 '/careers',
 'mailto:press@anthropic.com',
 'https://support.anthropic.com/',
 'https://status.anthropic.com/',
 '/supported-countries',
 'https://twitter.com/AnthropicAI',
 'https://www.linkedin.com/company/anthropicresearch',
 'https://www.youtube.com/@anthropic-ai',
 '/legal/consumer-terms',
 '/legal/commercial-terms',
 '/legal/privacy',
 '/legal/aup',
 '/responsible-disclosure-policy',
 'https://trust.anthropic.com/']

In [16]:
get_links("https://anthropic.com")

{'links': [{'type': 'Pagina Sobre nosotros',
   'url': 'https://anthropic.com/company'},
  {'type': 'Pagina de Carreras', 'url': 'https://anthropic.com/careers'},
  {'type': 'Pagina de Investigación', 'url': 'https://anthropic.com/research'},
  {'type': 'Pagina de Productos', 'url': 'https://anthropic.com/claude'}]}

In [ ]:
get_links("https://cursos.frogamesformacion.com")

## Segundo paso: ¡crea el folleto!

Reúne todos los detalles en otro mensaje para GPT4-o

In [19]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Links encontrados:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [20]:
print(get_all_details("https://anthropic.com"))

Links encontrados: {'links': [{'type': 'Página Acerca de', 'url': 'https://anthropic.com/company'}, {'type': 'Carreras', 'url': 'https://anthropic.com/careers'}, {'type': 'Página de Claude', 'url': 'https://www.anthropic.com/claude'}, {'type': 'Página de Investigación', 'url': 'https://anthropic.com/research'}, {'type': 'API', 'url': 'https://anthropic.com/api'}, {'type': 'Precios', 'url': 'https://anthropic.com/pricing'}, {'type': 'Página de Noticias', 'url': 'https://anthropic.com/news'}]}
Landing page:
Título de la Web:
Home \ Anthropic
Contenido de la Web:
Claude
Overview
Team
Enterprise
API
Pricing
Research
Company
Careers
News
AI
research
and
products
that put safety at the frontier
Claude.ai
Meet Claude 3.5 Sonnet
Claude 3.5 Sonnet, our most intelligent AI model, is now available.
Talk to Claude
API
Build with Claude
Start using Claude to drive efficiency and create new revenue streams.
Learn more
Announcements
Introducing computer use, a new Claude 3.5 Sonnet, and Claude 3.5 Ha

In [33]:
system_prompt = "Eres un asistente que analiza el contenido de varias páginas relevantes del sitio web de una empresa\
y crea un folleto breve sobre la empresa para posibles clientes, inversores y nuevos empleados. Responde en formato Markdown.\
Incluye detalles sobre la cultura de la empresa, los clientes, las carreras/empleos y los cursos/packs para futuros empleos si tienes la información."

# O descomenta las líneas a continuación para obtener un folleto más humorístico: esto demuestra lo fácil que es incorporar el "tono":

# system_prompt = "Eres un asistente que analiza el contenido de varias páginas relevantes del sitio web de una empresa \
# y crea un folleto breve, divertido y gracioso sobre la empresa para posibles clientes, inversores y nuevos empleados. Responde en formato Markdown.\
#Incluye detalles sobre la cultura de la empresa, los clientes y los cursos/packs para futuros empleos si tienes la información."


In [22]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"Estás mirando una empresa llamada: {company_name}\n"
    user_prompt += f"Aquí se encuentra el contenido de su página de inicio y otras páginas relevantes; usa esta información para crear un breve folleto de la empresa en Markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:20_000] # Truncar si tiene más de 20.000 caracteres
    return user_prompt

In [23]:
get_brochure_user_prompt("Anthropic", "https://anthropic.com")

Links encontrados: {'links': [{'type': 'Página Acerca de', 'url': 'https://www.anthropic.com/company'}, {'type': 'Página de Carreras', 'url': 'https://www.anthropic.com/careers'}, {'type': 'Página de Investigación', 'url': 'https://www.anthropic.com/research'}, {'type': 'Página de Claude', 'url': 'https://www.anthropic.com/claude'}, {'type': 'Página de API', 'url': 'https://www.anthropic.com/api'}, {'type': 'Página de Precios', 'url': 'https://www.anthropic.com/pricing'}, {'type': 'Página de Clientes', 'url': 'https://www.anthropic.com/customers'}]}


"Estás mirando una empresa llamada: Anthropic\nAquí se encuentra el contenido de su página de inicio y otras páginas relevantes; usa esta información para crear un breve folleto de la empresa en Markdown.\nLanding page:\nTítulo de la Web:\nHome \\ Anthropic\nContenido de la Web:\nClaude\nOverview\nTeam\nEnterprise\nAPI\nPricing\nResearch\nCompany\nCareers\nNews\nAI\nresearch\nand\nproducts\nthat put safety at the frontier\nClaude.ai\nMeet Claude 3.5 Sonnet\nClaude 3.5 Sonnet, our most intelligent AI model, is now available.\nTalk to Claude\nAPI\nBuild with Claude\nStart using Claude to drive efficiency and create new revenue streams.\nLearn more\nAnnouncements\nIntroducing computer use, a new Claude 3.5 Sonnet, and Claude 3.5 Haiku\nOct 22, 2024\nModel updates\n3.5 Sonnet\n3.5 Haiku\nOur Work\nProduct\nClaude for Enterprise\nSep 4, 2024\nAlignment\n·\nResearch\nConstitutional AI: Harmlessness from AI Feedback\nDec 15, 2022\nAnnouncements\nCore Views on AI Safety: When, Why, What, and H

In [24]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [25]:
create_brochure("Anthropic", "https://anthropic.com")

Links encontrados: {'links': [{'type': 'Página Acerca de', 'url': 'https://www.anthropic.com/company'}, {'type': 'Página de Carreras', 'url': 'https://www.anthropic.com/careers'}, {'type': 'Página de Investigación', 'url': 'https://www.anthropic.com/research'}, {'type': 'Página de Claude', 'url': 'https://www.anthropic.com/claude'}, {'type': 'Página de API', 'url': 'https://www.anthropic.com/api'}, {'type': 'Página de Precios', 'url': 'https://www.anthropic.com/pricing'}, {'type': 'Página de Noticias', 'url': 'https://www.anthropic.com/news'}]}


# Folleto de la Empresa: Anthropic

## **Acerca de Anthropic**
Anthropic es una empresa de seguridad e investigación en inteligencia artificial (IA) con sede en San Francisco. Nos dedicamos a construir sistemas AI confiables, interpretables y dirigibles. Creemos que la IA tendrá un impacto vasto en el mundo y estamos comprometidos a desarrollar herramientas que puedan ser utilizadas con confianza, al tiempo que investigamos las oportunidades y riesgos asociados con estas tecnologías.

### **Nuestra Misión**
Nuestra misión es asegurar que la IA transformadora ayude a las personas y a la sociedad a prosperar. En Anthropic, enfocamos nuestros esfuerzos en:

- **Investigación de frontera:** Realizamos investigaciones en diversos modelos y técnicas de seguridad, con el objetivo de crear sistemas que sean fáciles de entender y manejar.
- **Colaboración interdisciplinaria:** Contamos con un equipo diverso de investigadores, ingenieros y expertos en políticas, dedicados a la construcción de herramientas útiles y seguras.
- **Desarrollo responsable de IA:** Trabajamos junto a la sociedad civil, gobiernos y organizaciones académicas para promover prácticas de seguridad en la industria de la IA.

## **Cultura de la Empresa**
En Anthropic, fomentamos un entorno de **alta confianza**, donde valoramos la honestidad y la colaboración. Nuestros valores incluyen:

- **Estar aquí por la misión:** Nos unimos para avanzar en nuestros objetivos comunes.
- **Confianza inusual:** Asumimos buena fe y priorizamos la honestidad en todas nuestras interacciones.
- **Un solo gran equipo:** Colaboramos entre diferentes equipos para alcanzar nuestras metas.
- **Hacer lo sencillo que funciona:** Preferimos enfoques pragmáticos y basados en evidencia.

## **Productos y Clientes**
Nuestra principal herramienta es **Claude**, un modelo de IA avanzado que permite a empresas de múltiples industrias mejorar su eficiencia y crear nuevas fuentes de ingresos. Claude se caracteriza por:

- **Razonamiento avanzado:** Realiza tareas cognitivas complejas.
- **Análisis de visión:** Analiza imágenes y gráficos.
- **Generación de código:** Ayuda en la creación de software y sitios web.

Nuestros clientes incluyen organizaciones de diversos sectores que buscan integrar IA de manera efectiva y responsable.

## **Oportunidades de Carrera**
Anthropic está en búsqueda de individuos talentosos y apasionados para unirse a nuestro equipo. Ofrecemos un paquete completo de beneficios, que incluye:

- **Salud y bienestar:** Seguro médico integral, beneficios de fertilidad, y apoyo para salud mental.
- **Compensación competitiva:** Salarios ajustados al mercado y planes de acciones.
- **Flexibilidad:** Políticas de tiempo libre y trabajo remoto disponibles.

### **Proceso de Contratación**
Nuestro proceso de contratación está diseñado para evaluar tanto las habilidades técnicas como el encaje cultural, e incluye:

1. Revisión del currículum
2. Conversación exploratoria
3. Evaluación de habilidades (técnica o asignación para roles de operaciones)
4. Charla con el equipo
5. Entrevistas finales

**Oferta de Empleo:** Si seleccionamos a un candidato, le haremos una oferta formal, incluyendo opciones de desplazamiento, si es necesario.

## **Formación y Desarrollo**
Fomentamos el crecimiento de nuestros empleados mediante:

- **Estipendios educativos anuales.**
- **Apoyo para la instalación de oficinas en casa.**
- **Resolución de dudas sobre la IA y su uso en entornos empresariales.**

## **Conclusión**
Anthropic no solo se dedica a avanzar en la IA, sino que también busca garantizar que la tecnología se utilice de manera segura y efectiva. Si te apasiona la IA y quieres formar parte de un equipo que prioriza la seguridad y la ética, ¡te invitamos a considerar a Anthropic como tu próximo destino profesional!

**Descubre más en [Anthropic](https://www.anthropic.com/)**

---

### **Contacto**
Para más información sobre oportunidades laborales o nuestras soluciones de IA, visita nuestra página web o contáctanos directamente.

In [ ]:
create_brochure("Frogames Formación", "https://cursos.frogamesformacion.com")

## Por último, una pequeña mejora

Con un pequeño ajuste, podemos cambiar esto para que los resultados se transmitan desde OpenAI,
con la animación de máquina de escribir habitual


In [32]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [ ]:
stream_brochure("Anthropic", "https://anthropic.com")

## Aplicaciones empresariales

En este ejercicio, ampliamos el código del día 1 para realizar múltiples llamadas a LLM y generar un documento.

En términos de técnicas, este es quizás el primer ejemplo de patrones de diseño de Agentic AI, ya que combinamos múltiples llamadas a LLM. Esto se abordará más en la semana 2 y luego volveremos a Agentic AI de manera importante en la semana 8, cuando construyamos una solución Agent completamente autónoma.

En términos de aplicaciones empresariales, generar contenido de esta manera es uno de los casos de uso más comunes. Al igual que con el resumen, esto se puede aplicar a cualquier vertical empresarial. Escriba contenido de marketing, genere un tutorial de producto a partir de una especificación, cree contenido de correo electrónico personalizado y mucho más. Explore cómo puede aplicar la generación de contenido a su negocio e intente crear un prototipo de prueba de concepto.